# pytorchで簡単にニューラルネットワークを作ろう

## ニューラルネットワークとは

- できることを見ると
  - 回帰問題（数値予測）と分類問題が解けるやつ
  - 入力と期待される出力の組（教師データ）を与えればいい感じに補間してくれるやつ
    - 損失関数は上手に設定する必要がある
- 構成するパーツを見ると
  - 人工ニューロンがいっぱい繋がったやつ

## 人工ニューロンとは（数式は覚えなくていい）

- $d$ 個の入力（=入力ベクトル）$\boldsymbol{x}=(1,x_1,\dots,x_d)^{T}$ を受け、
  重み付け和を計算し、適当な関数（活性化関数）に通して出力するやつ
  - 単に内積で表せるようにダミーの $1$ を第0次元に付け足している（cf. 同次座標系）
- ニューロンの持つパラメータは
  - 重み $\boldsymbol{w}=(w_0,w_1,\dots,w_d)^{T}$
  - 活性化関数 $f$
    - しばしば同じ層のニューロンは同じ活性化関数を使う
- 出力は $f(\boldsymbol{w}^{T}\boldsymbol{x})=f(w_0+w_1x_1+\dots+w_dx_d)$

![ニューラルネットワークは多数のニューロンが接続されている](./resources/figures/neural_network.png)
![ニューロンの入出力](./resources/figures/artificial_neuron.png)

## ニューラルネットワークの万能近似定理（普遍性定理）

3層ニューラルネットワークはいくつかの条件のもと、その重みを調節することによって出力の誤差を任意の正数まで減らすことができる

現在は活性化関数はSigmoidalな関数でなくても良いとされている

## ニューラルネットワークで分類問題を解くには

$c$ クラス分類問題を解くならば、$c$ 出力のニューラルネットワークにする

入力を与えてForward計算をしたとき、出力層において最大の値を取っているノードのクラスに分類されたとする

## 画像分類問題を解く

### CIFAR-10とは

32x32サイズのRGBカラー画像が、10クラス各6000個用意されたデータセット。

### ニューラルネットワークに画像を入力する（一般論として）

サイズ $A \times B$、3チャンネル（RGB）固定であるとするならば、flattenして $3AB$ 次元のベクトルとして入力すれば良い

### CIFAR-10の場合

32x32 のRGB画像なので、入力は32x32x3=3072次元とする

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np

# CNN の定義
class CNN(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()

        # モデルの重みの定義
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=3)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3)
        # self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(in_features=(8*8*8), out_features=10)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # 入力から出力を計算する
        x = self.conv1(x)  # (30, 30, 16)
        x = self.relu(x)
        x = self.pool1(x)  # (10, 10, 16)
        x = self.conv2(x)  # (8, 8, 8)
        x = self.relu(x)
        # x = self.pool2(x)  # (4, 4, 8)
        x = self.flatten(x)  # (8*8*8,)
        x = self.fc(x)  # (10,)
        return x


def train_loop(model, dataloader, loss_fn, optimizer) -> None:
    # 学習用関数の定義
    num_classes = 10
    size = len(dataloader.dataset)
    for batch, (x, y) in enumerate(dataloader):
        pred = model(x)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

        
def test_loop(model, dataloader, loss_fn) -> None:
    # 評価用関数の定義
    num_classes = 10
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    confusion_matrix = np.zeros((num_classes, num_classes), dtype=int)
    
    with torch.no_grad():
        for x, y in dataloader:
            pred = model(x)
            loss = loss_fn(pred, y)
            test_loss += loss.item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            confusion_matrix[pred.argmax(1).detach().numpy(), y] += 1
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(confusion_matrix)
        

if __name__ == '__main__':
    # CIFAR-10 データセットを読み込む
    cifar10_data_train = torchvision.datasets.CIFAR10('./', train=True,  download=True, transform=torchvision.transforms.ToTensor())
    cifar10_data_test  = torchvision.datasets.CIFAR10('./', train=False, download=True, transform=torchvision.transforms.ToTensor())
    # DataLoader の設定
    dataloader_train = torch.utils.data.DataLoader(cifar10_data_train, batch_size=50, shuffle=True)
    dataloader_test  = torch.utils.data.DataLoader(cifar10_data_test,  batch_size=50, shuffle=True)
    
    cnn = CNN()
    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(cnn.parameters())
    
    epochs = 10
    for i in range(epochs):
        print(f"Epoch {i+1}\n-------------------------------")
        train_loop(cnn, dataloader_train, loss_fn, optimizer)
        test_loop(cnn, dataloader_test, loss_fn)

Files already downloaded and verified
Files already downloaded and verified
Epoch 1
-------------------------------
loss: 2.302583  [    0/50000]
loss: 2.127244  [ 5000/50000]
loss: 2.018109  [10000/50000]
loss: 1.816076  [15000/50000]
loss: 1.847676  [20000/50000]
loss: 1.511767  [25000/50000]
loss: 1.423052  [30000/50000]
loss: 1.619048  [35000/50000]
loss: 1.817378  [40000/50000]
loss: 1.490311  [45000/50000]
Test Error: 
 Accuracy: 45.4%, Avg loss: 1.539818 

[[181  28  78  23  29  16   9  17  89  34]
 [ 72 197  41  54  27  30  51  23  83 158]
 [ 24   6 123  41  67  24  30   9   6   6]
 [ 33  21  84 167  79 112  94  67  30  26]
 [  5   0  83  19 143  27  43  29   2   2]
 [ 24   7 116 142 103 175  52  82  13   8]
 [ 20  24  67  77  89  58 184  42   4  40]
 [ 42  31 101 105 135 131  88 195  20  48]
 [141  70  48  26  24  24  15  19 193  93]
 [ 33  61  13  30   5   7  21  38  32 174]]
Epoch 2
-------------------------------
loss: 1.520112  [    0/50000]
loss: 1.598086  [ 5000/50000]
l

## リバーシAIへの応用

* $400$ 入力、$400$ 出力のニューラルネットワークとして
* 教師データの与え方は？
  * 手で全部作る
  * 人間と戦わせて作る（上とほぼ同義だが）
  * アルゴリズムAIと戦わせて作る
  * 自身や前の世代の自身と戦わせて作る
* 全結合か、CNNか
  * CNNで事足りるかわからない
* 水増し、計算量削減
  * 回転や反転、平行移動で得られる盤面とそのときの手を学習させる
  * 盤面を何らかのアルゴリズムで正規化して回転や反転で得られる盤面を1つの状態に集約する